In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.getcwd()

'/Users/samardeep/Desktop/Energy Forecast/Scripts'

In [3]:
# Reading the data 
data_2015_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2015_Jul_Dec.csv")
data_2016_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2016_Jan_Jun.csv")
data_2016_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2016_Jul_Dec.csv")
data_2017_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2017_Jan_Jun.csv")
data_2017_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2017_Jul_Dec.csv")
data_2018_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2018_Jan_Jun.csv")
data_2018_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2018_Jul_Dec.csv")
data_2019_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2019_Jan_Jun.csv")
data_2019_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2019_Jul_Dec.csv")
data_2020_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2020_Jan_Jun.csv")
data_2020_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2020_Jul_Dec.csv")
data_2021_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2021_Jan_Jun.csv")
data_2021_july_dec = pd.read_csv("../Data/EIA930_BALANCE_2021_Jul_Dec.csv")
data_2022_jan_jun = pd.read_csv("../Data/EIA930_BALANCE_2022_Jan_Jun.csv")

data_2021_july_dec.head()

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources,Region
0,AEC,07/01/2021,1,07/01/2021 1:00:00 AM,07/01/2021 6:00:00 AM,915,472,461,-11,-11,...,-1,457,NaN,NaN,1,NaN,NaN,4,NaN,SE
1,AEC,07/01/2021,2,07/01/2021 2:00:00 AM,07/01/2021 7:00:00 AM,846,445,461,16,16,...,0,455,NaN,NaN,1,NaN,NaN,5,NaN,SE
2,AEC,07/01/2021,3,07/01/2021 3:00:00 AM,07/01/2021 8:00:00 AM,814,426,461,35,35,...,-1,457,NaN,NaN,1,NaN,NaN,4,NaN,SE
3,AEC,07/01/2021,4,07/01/2021 4:00:00 AM,07/01/2021 9:00:00 AM,792,426,460,34,34,...,0,455,NaN,NaN,1,NaN,NaN,5,NaN,SE
4,AEC,07/01/2021,5,07/01/2021 5:00:00 AM,07/01/2021 10:00:00 AM,794,422,460,38,38,...,-1,456,NaN,NaN,1,NaN,NaN,5,NaN,SE


In [4]:
# Creating a list to iterate over to generate region wise dataframes
data_lst = [data_2015_july_dec, data_2016_jan_jun, data_2016_july_dec, data_2017_jan_jun, data_2017_july_dec, 
            data_2018_jan_jun, data_2018_july_dec, data_2019_jan_jun, data_2019_july_dec, data_2020_jan_jun,
            data_2020_july_dec, data_2021_jan_jun, data_2021_july_dec, data_2022_jan_jun]

# Filtering the necessary features
for i in range(len(data_lst)):
    data_lst[i] = data_lst[i].loc[:, ['Balancing Authority', 'UTC Time at End of Hour', 'Demand (MW)', 'Net Generation (MW)', 'Region']]

In [5]:
# Converting the target feature values to  float data type and the UTC time column to datetime
cols_to_float = ['Demand (MW)', 'Net Generation (MW)']

for i in range(len(data_lst)):
    for column in cols_to_float:
        data_lst[i][column] = data_lst[i][column].str.replace(',', '').astype('float')
    data_lst[i]['UTC Time at End of Hour'] = pd.to_datetime(data_lst[i]['UTC Time at End of Hour'])
    data_lst[i].set_index('UTC Time at End of Hour', inplace = True) # set date column as index

In [6]:
# Combine dataframes from different time periods to create dataframes for each of the 13 regions
lst_regions_df = []
for data in data_lst:
    regions_df = [value for key, value in data.groupby('Region')]
    lst_regions_df.append(regions_df)

cal_df = pd.concat([lst_regions_df[i][0] for i in range(14)], axis = 0, ignore_index = False)
car_df = pd.concat([lst_regions_df[i][1] for i in range(14)], axis = 0, ignore_index = False)
cent_df = pd.concat([lst_regions_df[i][2] for i in range(14)], axis = 0, ignore_index = False)
fla_df = pd.concat([lst_regions_df[i][3] for i in range(14)], axis = 0, ignore_index = False)
mida_df = pd.concat([lst_regions_df[i][4] for i in range(14)], axis = 0, ignore_index = False)
midw_df = pd.concat([lst_regions_df[i][5] for i in range(14)], axis = 0, ignore_index = False)
ne_df = pd.concat([lst_regions_df[i][6] for i in range(14)], axis = 0, ignore_index = False)
nw_df = pd.concat([lst_regions_df[i][7] for i in range(14)], axis = 0, ignore_index = False)
ny_df = pd.concat([lst_regions_df[i][8] for i in range(14)], axis = 0, ignore_index = False)
se_df = pd.concat([lst_regions_df[i][9] for i in range(14)], axis = 0, ignore_index = False)
sw_df = pd.concat([lst_regions_df[i][10] for i in range(14)], axis = 0, ignore_index = False)
ten_df = pd.concat([lst_regions_df[i][11] for i in range(14)], axis = 0, ignore_index = False)
tex_df = pd.concat([lst_regions_df[i][12] for i in range(14)], axis = 0, ignore_index = False)

In [7]:
combined_regions_df = [cal_df, car_df, cent_df, fla_df, mida_df, midw_df, ne_df, nw_df,
                       ny_df, se_df, sw_df, ten_df, tex_df]

# Get the month and year for each UTC date
for df in combined_regions_df:
    df['Month'] = df.index.month
    df['Year'] = df.index.year

# Drop the data from March 2022 values
for df in combined_regions_df:
    index_to_drop = df.loc[(df['Month']==3)&(df['Year']==2022)].index
    df.drop(index_to_drop, axis = 0, inplace = True)

In [8]:
# Preprocess data

def drop_neg_values(df, col):
    ids_to_drop_neg_value = df.loc[(df[col] < 0)].index
    df.drop(ids_to_drop_neg_value, axis = 0, inplace = True) # Drop negative values
        
    df[col].fillna(method="ffill",inplace=True) # Forward fill null values
    return df

def outlier_capping(df, col):
    outlier_cap = np.percentile(df[col], 99) # Get 99th percentile value as outlier cap

    # Replace the outliers with the capped value
    for j in range(len(df[col])):
        if df[col][j] > outlier_cap:
            df[col][j] = outlier_cap
    return df


def preprocess_data(df_list):
    for i in range(len(df_list)):
        # Drop negative demand values
        df_list[i] = drop_neg_values(df_list[i], 'Demand (MW)')
        
        # Net Generation
        df_list[i] = drop_neg_values(df_list[i], 'Net Generation (MW)')


        df_list[i].drop(['Year', 'Month'], axis = 1, inplace = True) # Drop the year and month variables
        df_list[i] = df_list[i].resample('D').sum() # Resample data from hourly to daily

        # Handle demand outliers by capping values at 99th percentile
        df_list[i] = outlier_capping(df_list[i], 'Demand (MW)')

        # Handle net genration outliers by capping values at 99th percentile
        df_list[i] = outlier_capping(df_list[i], 'Net Generation (MW)')

    return df_list

In [9]:
combined_regions_df = preprocess_data(combined_regions_df)

In [10]:
# Save the preprocessed data to csv files
file_names = ["CAL.csv", "CAR.csv", "CENT.csv", "FLA.csv", "MIDA.csv", "MIDW.csv", "NE.csv", "NW.csv", "NY.csv", "SE.csv",
              "SW.csv", "TEN.csv", "TEX.csv"]

directory = "../Data/Preprocessed"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

for df, file_name in zip(combined_regions_df, file_names):
    file_path = os.path.join(directory, file_name)
    if not os.path.exists(file_path):
        df.to_csv(file_path, index_label= 'UTC Time at End of Hour')
        